# Guía No. 4 - Simulaciones de Monte Carlo
----
by: Diego Dorado

## Modelo de Ising 2D
----

Considere el modelo de Ising en la red cuadrada de $N = L \times L$ sitios (parámetro de red unitario) con interacciones entre primeros vecinos:

$$
H
=
-J\sum_{\langle i,j \rangle}S_iS_j
-
B\sum_iS_i
$$

Usando condiciones de contorno periódicas y tomando la energía en unidades de $k_B$ (esto es, tomando $k_B=1$) implemente un programa para simular las propiedades termodinámicas del modelo usando el algoritmo de Metropolis.

### Algoritmo de Metropolis

El algoritmo de Metropolis permite simular sistemas en equilibrio termodinámico mediante una cadena de configuraciones que respeta la distribución de Boltzmann.  
En el caso del modelo de Ising en 2D, una configuración corresponde al estado de todos los espines en la red.

1. **Inicialización de la configuración**: Elegir una configuración inicial arbitraria $ \alpha $.

    *Ising:* $ \alpha = (S_1, S_2, \dots, s_N)$, con $s_i = \pm 1$, representa todos los espines iniciales de la red.  

2. **Selección de un espín candidato**: Elegir una nueva configuración $\alpha'$ por algún método y se calcula $\Delta E = E(\alpha') - E(\alpha)$

    *Ising:* Elegir un sitio $ i $ de la red y proponer un cambio (flip) de espín: $ S_i \to -S_i $
     
   - El sitio puede elegirse:
       - **Secuencialmente:** recorrer todos los sitios en orden.
       - **Aleatoriamente:** seleccionar con probabilidad uniforme $ 1/N $.

3. **Criterio de aceptación**
   - Si $ \Delta E \leq 0 $, aceptar siempre el cambio.  
   - Si $ \Delta E > 0 $, aceptar el cambio con probabilidad: $p = e^{-\beta \Delta E}$ y se rechaza con probabilidad $1-p=1- e^{-\beta \Delta E}$.

    Para implementar esto en el código:  
     - Generar un número aleatorio $ r \in [0,1) $.  
     - Aceptar el cambio si $ r < p $.  

4. **Actualización de la configuración**
   - Si el cambio se acepta, la nueva configuración $ \alpha' $ pasa a ser el estado actual.  
   - Si se rechaza, se mantiene la configuración $ \alpha $.

5. **Repetición del proceso**
   - Repetir los pasos 2–5 muchas veces para generar una cadena de configuraciones.
   - Cada barrido de $ N $ intentos de flip se considera un **Monte Carlo step (MCS)**.

## Código - GPU
-----

### Paquetes
---

In [2]:
import Pkg
Pkg.add("CUDA")

    Updating registry at `C:\Users\diego\.julia\registries\General.toml`
   Resolving package versions...
   Installed NVTX_jll ─────────────── v3.2.2+0
   Installed LibTracyClient_jll ───── v0.9.1+6
   Installed Tracy ────────────────── v0.1.6
   Installed BFloat16s ────────────── v0.5.1
   Installed ScopedValues ─────────── v1.5.0
   Installed LLVMLoopInfo ─────────── v1.0.0
   Installed RandomNumbers ────────── v1.6.0
   Installed GPUArrays ────────────── v11.2.5
   Installed CUDA_Driver_jll ──────── v13.0.1+0
   Installed demumble_jll ─────────── v1.3.0+0
   Installed CUDA_Runtime_Discovery ─ v1.0.0
   Installed CUDA_Runtime_jll ─────── v0.19.1+0
   Installed CUDA_Compiler_jll ────── v0.2.1+0
   Installed HashArrayMappedTries ─── v0.2.0
   Installed Random123 ────────────── v1.7.1
   Installed NVTX ─────────────────── v1.0.1
   Installed GPUToolbox ───────────── v0.3.0
   Installed UnsafeAtomics ────────── v0.3.0
   Installed Atomix ───────────────── v1.1.2
   Installed LLVMExtra_j

In [3]:
using Statistics
using Random
using Plots
using LaTeXStrings
using CUDA

### Funciones

### Main
---

In [10]:
# -----------------------------
# Kernel de Metropolis (checkerboard)
# -----------------------------
function metropolis_kernel!(spins, L, T, J, B, parity)
    idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    total_sites = L * L

    if idx <= total_sites
        x = (idx - 1) % L + 1
        y = div(idx - 1, L) + 1

        if (x + y) % 2 == parity
            s = spins[x,y]

            # vecinos con condiciones periódicas
            xp = (x == L) ? 1 : x+1
            xm = (x == 1) ? L : x-1
            yp = (y == L) ? 1 : y+1
            ym = (y == 1) ? L : y-1

            h = J * s * (spins[xp,y] + spins[xm,y] + spins[x,yp] + spins[x,ym]) + B*s
            ΔE = 2h

            # probabilidad de flip
            if ΔE <= 0 || rand() < exp(-ΔE/T)
                spins[x,y] = -s
            end
        end
    end
    return
end

# -----------------------------
# Un sweep completo (GPU)
# -----------------------------
function sweep!(spins, L, T, J, B)
    threads = 256
    blocks = cld(L*L, threads)
    # Subred negra
    @cuda threads=threads blocks=blocks metropolis_kernel!(spins, L, T, J, B, 0)
    # Subred blanca
    @cuda threads=threads blocks=blocks metropolis_kernel!(spins, L, T, J, B, 1)
end

# -----------------------------
# Simulación Monte Carlo GPU
# -----------------------------
function run_MC_parallel(T, L; J=1.0, B=0.0, N_therm=5000, N_samples=10000)
    m_exp = zeros(length(T))  # magnetización promedio por temperatura

    for (i,t) in enumerate(T)
        # spins iniciales en GPU
        spins = CuArray(rand([-1,1], L, L))

        # Termalización
        for _ in 1:N_therm
            sweep!(spins, L, t, J, B)
        end

        # Muestreo
        m_samples = zeros(N_samples)
        for j in 1:N_samples
            sweep!(spins, L, t, J, B)
            # Magnetización total normalizada
            m_samples[j] = abs(CUDA.sum(spins)) / (L*L)
        end

        m_exp[i] = mean(m_samples)
    end

    return m_exp
end

run_MC_parallel (generic function with 1 method)

In [ ]:
# -----------------------------
# Ejecutar y graficar
# -----------------------------
L = 64
T = 1.0:0.5:4.0

m_exp = run_MC_parallel(T, L; N_therm=1000*L^2, N_samples=10000*L^2)

plot(T, m_exp, lw=2,
     xlabel="Temperatura", ylabel="Magnetización ⟨|M|⟩",
     title="Ising 2D paralelo en GPU")
